In [1]:
import json

In [3]:
raw_data = []
with open("_correct.json", 'r') as file:
    raw_data = json.load(file)
len(raw_data)

14745

In [69]:
def one_prompt(row: dict) -> str:
    user_prompt = (row['prompt']['user_prompt'])
    question, postfix = user_prompt.split("\n<postfix>\n")
    
    tmp = question.split("? at ")
    if len(tmp) > 1:
        question = tmp[0] + "?"
    
    postfix = postfix.split("\n<end>")[0]
    date, hours = postfix.split("\nrestaurant_hours:\n")
    date = date.split("current_date: ")[1]
    return (
        f"You are a helpful assistant in time management, especially in regards to restaurant hours.\n"
        f"When answering the question, consider the following at all times:\n"
        f"\t1. The name of the restaurant(s),\n"
        f"\t2. The time offered with appropriate modification if applicable,\n"
        f"\t3. The day of the week 'time' is. Use only the first three letters,\n"
        f"\t4. The opening hours of the restaurant(s) for the day,\n"
        f"\t5. Whether the restaurant(s) are open.\n"
        f"Your response to the final question should contain at least 20 tokens, and take your time to consider all the elements before your response.\n\n"
        f"Also, note the following before receiving the question:\n"
        f"Today's date: {date}\n"
        f"Restaurant hours:\n\n{hours}\n\n"
        f"Question: {question}"
    )

In [70]:
data = []
for i, row in enumerate(raw_data):
    data.append({
        'prompt': one_prompt(row),
        'output': row['full_response']['text']
    })

In [3]:
data = []
with open('data.json', 'r') as file:
    data = json.load(file)

In [22]:
print(data[3076+1755]['prompt'])

You are a helpful assistant in time management, especially in regards to restaurant hours.
When answering the question, consider the following at all times:
	1. The name of the restaurant(s),
	2. The time offered with appropriate modification if applicable,
	3. The day of the week 'time' is. Use only the first three letters,
	4. The opening hours of the restaurant(s) for the day,
	5. Whether the restaurant(s) are open.
Your response to the final question should contain at least 20 tokens, and take your time to consider all the elements before your response.

Also, note the following before receiving the question:
Today's date: December 13, 2030
Restaurant hours:

Chili Chili Bang Bang
Fri: 04:00AM-09:45AM, 03:45PM-07:45PM
Mon: 04:30AM-06:45PM
Sat: 12:30PM-10:45PM
Sun: 01:30PM-01:45PM
Thu: 01:15PM-11:15PM
Tue: 12:45AM-10:30AM, 05:00PM-10:45PM
Wed: 02:00AM-03:00AM, 03:15PM-04:15PM

Blue Sky Deli
Fri: 07:15AM-11:00AM, 09:45PM-11:45PM
Mon: 05:00AM-07:15AM
Sat: Closed
Sun: 03:30AM-10:15PM
T

In [21]:
len(data)

7365

In [22]:
with open("_correct-2.json", 'r') as file:
    data = json.load(file)
with open("_incorrect-2.json", 'r') as file:
    data += json.load(file)
len(data)

10000

In [23]:
data[0]

{'id': 2,
 'prompt': {'user_prompt': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Cl

In [24]:
sorted_list = sorted(data, key= lambda d: d['id'])

In [25]:
sorted_list[0]

{'id': 0,
 'prompt': {'user_prompt': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Cl

In [26]:
question_data = []
for row in sorted_list:
    r = row['prompt']
    del r['items_debug']
    del r['system_prompt']
    r['input'] = r.pop('user_prompt')
    r['output'] = r.pop('expected_response')
    question_data.append(r)

In [27]:
question_data[0]

{'input': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Closed",\nTake your time to c

In [31]:
tmp = "Here is the expected output: {\"day_of_week\": \"Fri\", \"is_open\": [\"Froggy's Fried Chicken\", \"Iron Chef Sushi Bar\"]}\n\nHere is the user input: The exact time currently is 12:00:00 AM, Oct 03, 2026. Which restaurants are open now?\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235"

In [32]:
count = 0
for c in tmp:
    if c == '\u7235':
        count += 1
count

67

In [35]:
tmp = "The output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n"
print(tmp)

The output should be:
    {
        "day_of_week": "Wed",
        "is_open": [
            "The Pantry",
            "Ocean Breeze Bistro",
            "Cactus Cantina",
            "The Hippie Hut"
        ]
    }

Here is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.爵士assistant爵士

The output should be:
    {
        "day_of_week": "Wed",
        "is_open": [
            "The Pantry",
            "Ocean Breeze Bistro",
            "Cactus Cantina",
            "The Hippie Hut"
        ]
    }

Here is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.爵士assistant爵士

The output should be:
    {
        "day_of_week": "Wed",
        "is_open": [
            "The Pantry",



In [ ]:
datum = {
    "id": 1,
    "prompt": {
        "user_prompt": "Output the following in order given the user question and a database of restaurant hours:\n    1. 'day_of_week': the day of the week \"time\" is. Use only the first three letters,\n    2. 'is_open': a list of all open restaurants, in the following format: [\"open_restaurant_name\", ...]\nBefore responding, consider the following items:\n    3. 'time': the time offered with appropriate modification if applicable, in the following format: 'YYYY-MM-DD hh:mmXM',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today's date (earliest being '1970-01-01 12:00AM'),\n        b. Completely disregard timezones for now,\n    4. 'opening_hours': the opening hours for the day, in the following format: {\"restaurant_name\": \"XX:XXAM-XX:XXPM\", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write \"Closed\",\nTake your time to consider all the elements, and your answers should be consistent.\nHere is a database of all the restaurants to consider for this task:\n\nThe Pantry\nFri: 02:00AM-08:30AM, 07:00PM-09:00PM\nMon: 01:30PM-01:45PM, 03:15PM-06:00PM\nSat: 12:00PM-10:15PM\nSun: 01:15AM-10:15AM, 04:15PM-06:00PM\nThu: 03:15AM-04:30AM, 05:30PM-08:00PM\nTue: 01:00AM-10:15AM, 07:15PM-09:45PM\nWed: 04:15PM-08:15PM\n\nOcean Breeze Bistro\nFri: 03:45AM-04:45AM\nMon: 03:45AM-10:15PM\nSat: 07:15AM-10:45AM, 05:45PM-11:45PM\nSun: 03:15AM-08:30AM, 07:15PM-09:30PM\nThu: 02:00AM-01:15PM, 04:30PM-06:45PM\nTue: 12:00AM-05:00AM\nWed: 06:45AM-07:30AM, 04:00PM-08:15PM\n\nCactus Cantina\nFri: 07:15AM-12:15PM\nMon: 12:15AM-06:30AM, 03:45PM-11:45PM\nSat: 06:45AM-12:00PM, 04:00PM-09:00PM\nSun: 12:15AM-10:30AM, 06:30PM-10:00PM\nThu: 03:45PM-07:00PM\nTue: 10:45AM-11:00AM, 05:15PM-08:15PM\nWed: 03:30AM-04:45AM\n\nThe Hippie Hut\nFri: 01:45AM-09:15AM\nMon: 10:00AM-11:30PM\nSat: 01:00AM-03:45AM, 02:00PM-08:45PM\nSun: 03:45AM-06:15PM\nThu: 03:00AM-03:15AM, 04:30PM-07:00PM\nTue: 02:15AM-06:30AM, 02:15PM-10:30PM\nWed: 12:15AM-10:30AM\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.",
        "system_prompt": "You are a helpful assistant in time management, especially in regards to restaurant hours. Always output in JSON",
        "expected_response": {
            "day_of_week": "Wed",
            "is_open": [
                "The Hippie Hut"
            ]
        },
    },
    "full_response": "The output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n"
}

In [9]:
with open("data.json", 'w') as file:
    json.dump(data, file, indent=4)

In [6]:
for row in data:
    row['output'] = json.dumps(row['output'])

In [7]:
data[0]

{'input': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Closed",\nTake your time to c

In [4]:
with open("hours.json", 'r') as file:
    data = json.load(file)
with open("hours.json", 'w') as file:
    json.dump(data, file, indent=4)

In [6]:
with open("data.json", 'r') as file:
    data = json.load(file)

with open("data.jsonl", 'w') as file:
    file.writelines([f"{json.dumps(datum)}\n" for datum in data])

In [9]:
raw_data[0]

{'id': 0,
 'prompt': {'problem_type': 'TO_LIST',
  'use_delta': False,
  'today': 'May 22, 2028',
  'opening_hours': {'Bamboo Garden Sushi Bar': {'Fri': '10:00AM-11:30AM, 05:45PM-07:30PM',
    'Mon': '04:15AM-02:00PM, 06:15PM-08:15PM',
    'Sat': '07:45AM-03:30PM',
    'Sun': '02:15AM-07:30AM',
    'Thu': '12:30AM-06:00AM',
    'Tue': '12:00PM-12:15PM',
    'Wed': '04:30PM-05:45PM'},
   'Cheese & Charcuterie': {'Fri': '07:45AM-08:00AM',
    'Mon': '01:15AM-11:15AM, 07:15PM-07:45PM',
    'Sat': '04:30PM-07:45PM',
    'Sun': '08:30AM-06:00PM',
    'Thu': '09:00AM-05:00PM',
    'Tue': '07:45AM-11:30AM',
    'Wed': '01:45AM-06:30AM, 03:45PM-08:00PM'},
   'Pasta Paradise': {'Fri': '05:45AM-06:15AM, 07:45PM-10:45PM',
    'Mon': '07:00AM-11:30PM',
    'Sat': '',
    'Sun': '08:00AM-12:00PM',
    'Thu': '04:30AM-02:00PM',
    'Tue': '12:00AM-07:00AM, 04:15PM-07:45PM',
    'Wed': '03:45AM-10:00PM'},
   'Pacific Rim Bistro': {'Fri': '',
    'Mon': '05:45AM-06:30AM',
    'Sat': '08:15AM-10:15AM, 

In [5]:
split_location = int(len(data) * 0.8)
split_location

11741

In [6]:
with open("train.jsonl", 'w') as file:
    file.writelines([f"{json.dumps(line)}\n" for line in data[:split_location]])
with open("test.jsonl", 'w') as file:
    file.writelines([f"{json.dumps(line)}\n" for line in data[split_location:]])

In [4]:
count = 0
data = []
for line in raw_data:
    if line['prompt']['expected_response']['day_of_week'] == line['full_response']['day_of_week']:
        count += 1
        line['prompt']['expected_response'] = line['full_response']
        data.append(line['prompt'])
count, len(raw_data)

(14677, 14745)

In [24]:
with open("train.jsonl", 'w') as file:
    file.write("Overridden!")

In [28]:
data[0]

{'problem_type': 'TO_LIST',
 'use_delta': False,
 'today': 'May 22, 2028',
 'opening_hours': {'Bamboo Garden Sushi Bar': {'Fri': '10:00AM-11:30AM, 05:45PM-07:30PM',
   'Mon': '04:15AM-02:00PM, 06:15PM-08:15PM',
   'Sat': '07:45AM-03:30PM',
   'Sun': '02:15AM-07:30AM',
   'Thu': '12:30AM-06:00AM',
   'Tue': '12:00PM-12:15PM',
   'Wed': '04:30PM-05:45PM'},
  'Cheese & Charcuterie': {'Fri': '07:45AM-08:00AM',
   'Mon': '01:15AM-11:15AM, 07:15PM-07:45PM',
   'Sat': '04:30PM-07:45PM',
   'Sun': '08:30AM-06:00PM',
   'Thu': '09:00AM-05:00PM',
   'Tue': '07:45AM-11:30AM',
   'Wed': '01:45AM-06:30AM, 03:45PM-08:00PM'},
  'Pasta Paradise': {'Fri': '05:45AM-06:15AM, 07:45PM-10:45PM',
   'Mon': '07:00AM-11:30PM',
   'Sat': '',
   'Sun': '08:00AM-12:00PM',
   'Thu': '04:30AM-02:00PM',
   'Tue': '12:00AM-07:00AM, 04:15PM-07:45PM',
   'Wed': '03:45AM-10:00PM'},
  'Pacific Rim Bistro': {'Fri': '',
   'Mon': '05:45AM-06:30AM',
   'Sat': '08:15AM-10:15AM, 06:30PM-11:45PM',
   'Sun': '',
   'Thu': '01:4

In [2]:
with open("hours.json") as file:
    hours_data = json.load(file)
hours_data

[{'name': 'Magnolia Gardens',
  'hours': {'Mon': '02:30AM-06:45AM, 04:15PM-11:15PM',
   'Tue': '01:45PM-02:30PM',
   'Wed': '02:30AM-05:45AM, 11:30AM-09:45PM',
   'Thu': '05:30AM-09:15AM, 01:00PM-05:45PM',
   'Fri': '12:45AM-11:15PM',
   'Sat': '05:15PM-08:45PM',
   'Sun': '04:30AM-01:00PM, 09:45PM-10:00PM'}},
 {'name': 'The Nutcracker Sweets Bakery',
  'hours': {'Mon': '10:00AM-02:30PM, 08:00PM-08:15PM',
   'Tue': '01:00PM-06:15PM',
   'Wed': '12:00PM-10:15PM',
   'Thu': '01:30AM-02:00PM',
   'Fri': '02:15AM-11:45PM',
   'Sat': '12:30AM-10:45AM, 02:30PM-11:00PM',
   'Sun': '12:30AM-01:45AM, 02:15PM-11:45PM'}},
 {'name': 'The Jubilee Juice Bar',
  'hours': {'Mon': '12:30AM-06:30AM, 07:15PM-11:30PM',
   'Tue': '12:15AM-03:30AM',
   'Wed': '05:15AM-07:15AM, 08:45PM-10:45PM',
   'Thu': '08:00AM-08:00PM',
   'Fri': '07:30AM-09:15PM',
   'Sat': '01:30AM-10:00AM, 08:00PM-10:30PM',
   'Sun': '03:30AM-05:30AM, 04:30PM-11:45PM'}},
 {'name': 'The Pita Pit',
  'hours': {'Mon': '03:00AM-08:15AM, 0

In [3]:
new_hours_data = {}
for row in hours_data:
    new_hours_data[row["name"]] = row["hours"]
new_hours_data

{'Magnolia Gardens': {'Mon': '02:30AM-06:45AM, 04:15PM-11:15PM',
  'Tue': '01:45PM-02:30PM',
  'Wed': '02:30AM-05:45AM, 11:30AM-09:45PM',
  'Thu': '05:30AM-09:15AM, 01:00PM-05:45PM',
  'Fri': '12:45AM-11:15PM',
  'Sat': '05:15PM-08:45PM',
  'Sun': '04:30AM-01:00PM, 09:45PM-10:00PM'},
 'The Nutcracker Sweets Bakery': {'Mon': '10:00AM-02:30PM, 08:00PM-08:15PM',
  'Tue': '01:00PM-06:15PM',
  'Wed': '12:00PM-10:15PM',
  'Thu': '01:30AM-02:00PM',
  'Fri': '02:15AM-11:45PM',
  'Sat': '12:30AM-10:45AM, 02:30PM-11:00PM',
  'Sun': '12:30AM-01:45AM, 02:15PM-11:45PM'},
 'The Jubilee Juice Bar': {'Mon': '12:30AM-06:30AM, 07:15PM-11:30PM',
  'Tue': '12:15AM-03:30AM',
  'Wed': '05:15AM-07:15AM, 08:45PM-10:45PM',
  'Thu': '08:00AM-08:00PM',
  'Fri': '07:30AM-09:15PM',
  'Sat': '01:30AM-10:00AM, 08:00PM-10:30PM',
  'Sun': '03:30AM-05:30AM, 04:30PM-11:45PM'},
 'The Pita Pit': {'Mon': '03:00AM-08:15AM, 03:30PM-05:00PM',
  'Tue': '12:45AM-08:15AM',
  'Wed': '12:45AM-07:15AM, 04:45PM-08:00PM',
  'Thu': '0

In [7]:
with open("hours.json", 'w') as file:
    json.dump(new_hours_data, file, indent=4)

In [20]:
print("{\n    \"time\": \"2024-09-29 10:27AM\",\n    \"day_of_week\": \"Sun\",\n    \"opening_hours\": {\n        \"Taste Odyssey\": \"05:30AM-05:00PM\",\n        \"Mirthful Munch\": \"07:00AM-08:00AM, 07:00PM-11:45PM\",\n        \"Divine Dips': \": \": \": \": \":\"\n    },\n    \"is_open\": [\n        \"Mirthful Munch\"\n    ]\n}")

{
    "time": "2024-09-29 10:27AM",
    "day_of_week": "Sun",
    "opening_hours": {
        "Taste Odyssey": "05:30AM-05:00PM",
        "Mirthful Munch": "07:00AM-08:00AM, 07:00PM-11:45PM",
        "Divine Dips': ": ": ": ": ":"
    },
    "is_open": [
        "Mirthful Munch"
    ]
}


In [16]:
with open("_correct.json", 'r') as file:
    correct_ds = json.load(file)

In [29]:
with open("_incorrect.json", 'r') as file:
    correct_ds = json.load(file)
count = 0
for row in correct_ds:
    if type(row['full_response']) is dict and row['prompt']['expected_response']['day_of_week'] == row['full_response']['day_of_week']:
        count += 1
count

490

In [102]:
with open("_incorrect.json", 'r') as file:
    incorrect_ds = json.load(file)
len(incorrect_ds)
ds = []
count = 0
for row in incorrect_ds:
    res = row['full_response']
    ans = row['prompt']['expected_response']
    if type(res) != dict or type(ans) != dict:
        continue
    for key, val in ans.items():
        if key in res and val == res[key]:
            del res[key]
        else:
            pass
            
    ds.append((ans, res))
print(json.dumps(ds, indent=4))

[
    [
        {
            "time": "2028-03-10 12:57PM",
            "day_of_week": "Fri",
            "opening_hours": {
                "Magical Morsels": "11:30AM-01:30PM",
                "Alcove Aroma": "01:30AM-07:30AM, 11:00AM-06:00PM",
                "Coastal Cravings": "01:45AM-12:00PM",
                "Bounty Bistro": "02:30AM-12:45PM, 03:00PM-09:30PM",
                "Treasure Diner": "03:30AM-07:00PM"
            },
            "is_open": [
                "Magical Morsels",
                "Alcove Aroma",
                "Treasure Diner"
            ]
        },
        {
            "is_open": [
                "Magical Morsels",
                "Alcove Aroma",
                "Trasure Diner"
            ],
            "reasoning": "The current time (2028-03-10 at 12:57 PM) falls within the operating hours of all three listed restaurants on Friday ('Fri'). Specifically, Magical Morsels operates from 11:30 AM to 01:30 PM; Alcove Aroma has two slots - one from 01:30 A

In [95]:
print("{\n    \"time\": \"2027-05-26 07:14PM\",\n    \"day_of_week\": \"Wed\",\n    \"opening_hours\": {\n        \"Savannah Supper\": \"09:30AM-06:00PM\",\n        \"The Hungry Spoon\": \"08:00AM-06:00PM\",\n        \"Serenade Spoon\": \"03:00AM-08:30AM, 08:00PM-09:30PM\",\n        \"Farmhouse Feast\": \"07:00AM-11:30AM, 08:30PM-10:45PM\",\n        \"Whimsical Wok\": \"02:45AM-02:00PM\"\n    },\n    \"is_open\": [],\n    \"closed_at\": [\n        \"None\"\n    ],\n    \"open later at\": [\n        \"At exactly 7:14 PM on Wednesday (May 26, 2027), none of the listed restaurants were open. All their operating hours do not cover this time period.\"\n}")

{
    "time": "2027-05-26 07:14PM",
    "day_of_week": "Wed",
    "opening_hours": {
        "Savannah Supper": "09:30AM-06:00PM",
        "The Hungry Spoon": "08:00AM-06:00PM",
        "Serenade Spoon": "03:00AM-08:30AM, 08:00PM-09:30PM",
        "Farmhouse Feast": "07:00AM-11:30AM, 08:30PM-10:45PM",
        "Whimsical Wok": "02:45AM-02:00PM"
    },
    "is_open": [],
    "closed_at": [
        "None"
    ],
    "open later at": [
        "At exactly 7:14 PM on Wednesday (May 26, 2027), none of the listed restaurants were open. All their operating hours do not cover this time period."
}


In [39]:
a[0]+a[1], a[2]+a[3]

(0.4789977072513639, 0.5210022927486361)

In [60]:
tp = 10000
tn = 2000
fp = 3000
fn = 4000
def print_report(title: str, tp: int, fp: int, fn: int, tn: int):
    total = tp + tn + fp + fn
    ep = tp + fn
    en = fp + tn
    rp = tp + fp
    rn = tn + fn
    pprec = tp / rp * 100
    precl = tp / ep * 100
    nprec = tn / rn * 100
    nrecl = tn / en * 100
    pf1 = 2/(1/pprec+1/precl)
    nf1 = 2/(1/nprec+1/nrecl)

    out = ''
    with open("report_template.txt", 'r') as file:
        out = file.read().format(
            title=title,
            tp=tp, tn=tn, fp=fp, fn=fn, total=total,
            ep=ep, en=en, rp=rp, rn=rn,
            pprec=pprec, nprec=nprec, precl=precl, nrecl=nrecl,
            pf1=pf1, nf1=nf1
        )

    with open("_report.out", 'w') as file:
        file.write(out)

print_report("output", 1000, 0, 0, 1000)